## Analyzing Inland Empire Patents with Python
*A study on visualizing patent localization through data manipulation.*

#### Goals:

This notebook was built with three foundational goals in mind:

1. **Parse** a pre-existing dataset of patents in the United States, limiting our new dataset to patents filed in the Inland Empire.
2. **Create** a new clean dataset, discarding fields we don't need, and adding fields that we want.
3. **Visualize** our patent data onto a map using ArcGIS.

#### Downloading our Data:

We are solely using [Pandas](https://pandas.pydata.org/) for joining and manipulating data across different `.csv` files

In [2]:
# importing our libraries
import os
import pandas as pd

First let's download our main datasets from the US Patent and Trademark Office (USPTO)

In [3]:
# these commands will take several minutes
!wget -cq https://bulkdata.uspto.gov/data/patent/pair/economics/2021/all_inventors.csv.zip
!wget -cq https://bulkdata.uspto.gov/data/patent/pair/economics/2021/application_data.csv.zip

!unzip -qq all_inventors.csv.zip
!unzip -qq application_data.csv.zip

We also have two additional datasets to download, we will see what they are used for later

In [4]:
!git clone https://github.com/jonahbrawley/ie-patents
!mv /content/ie-patents/*.csv /content

# delete unneeded files/folders
!rm -rf ie-patents *.zip
!rm -rf /content/sample_data/*.* sample_data

Cloning into 'ie-patents'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), done.


#### Loading the Datasets

Now that we have all our data downloaded, let's start loading it all into DataFrames. The datasets we will primarily work with are made by the USPTO, and include data on patents filed between Jan 1970 and Dec 2021.

I will now assign some helpful variables using a `source_contents` naming scheme.
- `source` is pretty self explanatory and is the name of the dataset source
- `contents` being `path` or `size` respectively, which refers to either file path or size in bytes

In [12]:
# importing relevant CSV (and helpful information)
temp = "/content/%s.csv"

appdata_path = temp % "application_data" # csv with individual application data (3.2GB)
appdata_size = str(os.path.getsize(appdata_path))

uspto_path = temp % "all_inventors" # csv with inventor information (1.3GB)
uspto_size = str(os.path.getsize(uspto_path))

print(f"{uspto_size} bytes {uspto_path}\n{appdata_size} bytes {appdata_path}")

1389095073 bytes /content/all_inventors.csv
3308402654 bytes /content/application_data.csv


We also need to know the names of all the cities in the Inland Empire, so I've created `ie_cities.csv` with those values. Our US city coordinates are located in a separate file named `coordinates.csv`.

In [13]:
cities_path = temp % "ie_cities" # custom csv with names of the Inland Empire cities (2KB)
print(cities_path)

coord_path = temp % "coordinates" # csv with coordinates of major US cities (4.5MB)
print(coord_path)

/content/ie_cities.csv
/content/coordinates.csv


Now, let's convert our datasets into a DataFrames! Our motive is to begin by parsing the USPTO data, and then to limit that to just the Inland Empire (referred to as IE moving forward.) We only want `application_number` and `filing_date` from `application_data.csv`, so because it's such a huge file we will only load these columns to be resource efficient.

These commands will take several minutes.

In [14]:
# load USPTO inventor information
uspto_df = pd.read_csv(uspto_path)
print(uspto_df)

# load USPTO application information
appdata_df = pd.read_csv(appdata_path, usecols=["application_number","filing_date"])
print(appdata_df)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


         application_number inventor_name_first inventor_name_middle  \
0                  04840815             WILLIAM                   D.   
1                  05434252              HERMAN                  NaN   
2                  05434252                ERIC                   K.   
3                  05603052                AGIS                   F.   
4                  05823902             WILLIAM                   G.   
...                     ...                 ...                  ...   
32549781           17594783             Zhihuan                  NaN   
32549782           17594783            Weisheng                  NaN   
32549783           17438358                Toru                  NaN   
32549784           17438358              Koyuru                  NaN   
32549785           17596594              Martin                  NaN   

         inventor_name_last  inventor_rank  inventor_city_name  \
0                 SCHAEFFER              1              POMONA   
1  

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


         application_number filing_date
0                  04453098         NaN
1                  04569423         NaN
2                  04577552         NaN
3                  04589670         NaN
4                  04589018         NaN
...                     ...         ...
13556604           17781537  2022-06-01
13556605           17613278  2022-05-31
13556606           17829197  2022-05-31
13556607           35514482  2022-05-31
13556608           17597405  2022-06-13

[13556609 rows x 2 columns]


Now let's load our additional data into DataFrames...

In [15]:
# load IE city names
cities_df = pd.read_csv(cities_path)
print(cities_df)

# load US city coordinates
coord_df = pd.read_csv(coord_path)
print(coord_df)

     inventor_city_name
0        Lucerne Valley
1             Riverside
2              Temecula
3                Corona
4              Murrieta
..                  ...
105            Fawnskin
106               Yermo
107             Aguanga
108  North Palm Springs
109             Daggett

[110 rows x 1 columns]
              city   city_ascii state_id  state_name  county_fips  \
0         New York     New York       NY    New York        36061   
1      Los Angeles  Los Angeles       CA  California         6037   
2          Chicago      Chicago       IL    Illinois        17031   
3            Miami        Miami       FL     Florida        12086   
4           Dallas       Dallas       TX       Texas        48113   
...            ...          ...      ...         ...          ...   
28333        Gross        Gross       NE    Nebraska        31015   
28334       Lotsee       Lotsee       OK    Oklahoma        40143   
28335    The Ranch    The Ranch       MN   Minnesota        27087  

#### Parsing the Set

Now that all our data is ready, let's begin the conversion process. We need to first decide what to do with any missing data fields.
There are four fields we must require:
1. `application_number`
2. `inventor_city_name`
3. `inventor_region_code`
4. `inventor_country_code`

In [16]:
# drop rows missing any of these fields
uspto_df.dropna(subset=["application_number", "inventor_city_name", "inventor_region_code", "inventor_country_code"], inplace=True)

Let's also remove rows for patents not located in California, USA.

In [17]:
# drop patents not in CA, USA
uspto_df.drop(uspto_df.query(" `inventor_country_code`!='US' or `inventor_region_code`!='CA' ").index, inplace=True)

# test if theyre gone (our output should be blank)
uspto_df.query("`inventor_region_code`!='CA'")

,application_number,inventor_name_first,inventor_name_middle,inventor_name_last,inventor_rank,inventor_city_name,inventor_region_code,inventor_country_code


Now we should convert our city names to align with `uspto_df`. Then with a simple merge, we can build a new dataframe of only cities in the IE.

In [18]:
# change capitalization
uspto_df["inventor_city_name"] = uspto_df["inventor_city_name"].str.upper()
cities_df["inventor_city_name"] = cities_df["inventor_city_name"].str.upper()

# merge cities to a new df (only cities in IE)
ie_df = pd.merge(uspto_df, cities_df, on=["inventor_city_name"], how="inner")

ie_df.head()

,application_number,inventor_name_first,inventor_name_middle,inventor_name_last,inventor_rank,inventor_city_name,inventor_region_code,inventor_country_code
0,10545256,Bent,L,Nielsen,1,RIALTO,CA,US
1,04643297,WILLIAM,R.,MAGORIAN,1,RIALTO,CA,US
2,6050185,RONALD,G.,FISHERING,1,RIALTO,CA,US
3,6050185,PETER,M.,FISHERING,2,RIALTO,CA,US
4,6120673,RICHARD,D.,BOLDING,1,RIALTO,CA,US


This code below shows all the unique entries in `inventor_city_name`. If our methods worked, it should display only IE cities.

In [19]:
ie_df.inventor_city_name.unique()

array(['RIALTO', 'TEMECULA', 'CHINO HILLS', 'CORONA', 'MURRIETA',
       'MIRA LOMA', 'UPLAND', 'RIVERSIDE', 'PALM DESERT', 'INDIAN WELLS',
       'ONTARIO', 'REDLANDS', 'LA QUINTA', 'NORCO', 'CHINO',
       'SAN BERNARDINO', 'BIG BEAR CITY', 'COLTON', 'HEMET', 'YUCAIPA',
       'PALM SPRINGS', 'YUCCA VALLEY', 'FONTANA', 'LAKE ELSINORE',
       'LAKE ARROWHEAD', 'MONTCLAIR', 'APPLE VALLEY', 'CALIMESA',
       'WRIGHTWOOD', 'BANNING', 'LOMA LINDA', 'HIGHLAND', 'PERRIS',
       'BLYTHE', 'HESPERIA', 'SAN JACINTO', 'RANCHO MIRAGE', 'SUN CITY',
       'CRESTLINE', 'BLUE JAY', 'VICTORVILLE', 'HELENDALE',
       'CATHEDRAL CITY', 'BARSTOW', 'BIG BEAR LAKE', 'HOMELAND',
       'CEDAR GLEN', 'BEAUMONT', 'FAWNSKIN', 'IDYLLWILD', 'ANZA',
       'PHELAN', 'MENTONE', 'TRONA', 'THERMAL', 'ADELANTO', 'SUGARLOAF',
       'INDIO', 'WINCHESTER', 'GRAND TERRACE', 'BLOOMINGTON',
       'ORO GRANDE', 'LANDERS', 'MOUNTAIN CENTER', 'RUNNING SPRINGS',
       'WILDOMAR', 'MORENO VALLEY', 'FOREST FALLS', 'AGUA

#### Cleaning the Set and Adding Features

So far we have successfully limited our set to just IE cites. Let's proceed to cleaning data we don't need, and then adding in fields that we do. We don't have any use for `inventor_rank`, so lets start with that.

In [20]:
ie_df.drop("inventor_rank", inplace=True, axis=1)

We need to now use `appdata_df` to add the `filing_date` field to `ie_df`. We can do this using `join()`, which we will dive into more later.

In [21]:
ie_df = ie_df.join(appdata_df.set_index("application_number"), on="application_number")
ie_df.dropna(subset=["filing_date"], inplace=True) # drop any rows if filing_date data is missing
ie_df.head()

,application_number,inventor_name_first,inventor_name_middle,inventor_name_last,inventor_city_name,inventor_region_code,inventor_country_code,filing_date
1,04643297,WILLIAM,R.,MAGORIAN,RIALTO,CA,US,1967-05-25
4,6120673,RICHARD,D.,BOLDING,RIALTO,CA,US,1980-02-11
5,6139628,RAYMOND,A.,ELLESTAD,RIALTO,CA,US,1980-04-14
6,6188826,WILLIAM,S.,BOKON,RIALTO,CA,US,1980-09-19
7,6200992,GORDON,A.,AUDET,RIALTO,CA,US,1980-10-27


Let's filter rows out of our US city coordinates DataFrame to limit it to IE city coordinates.

In [22]:
coord_df["city"] = coord_df["city"].str.upper() # convert city column to uppercase
iecoord_df = coord_df.merge(cities_df, how="right", left_on="city", right_on="inventor_city_name")

iecoord_df.drop(iecoord_df.query(" `state_id`!='CA' ").index, inplace=True)
print(iecoord_df)

               city      city_ascii state_id  state_name  county_fips  \
0    LUCERNE VALLEY  Lucerne Valley       CA  California       6071.0   
1         RIVERSIDE       Riverside       CA  California       6065.0   
17         TEMECULA        Temecula       CA  California       6065.0   
18           CORONA          Corona       CA  California       6065.0   
21         MURRIETA        Murrieta       CA  California       6065.0   
..              ...             ...      ...         ...          ...   
166           MECCA           Mecca       CA  California       6065.0   
173      WINCHESTER      Winchester       CA  California       6065.0   
187      WRIGHTWOOD      Wrightwood       CA  California       6071.0   
189      FORT IRWIN      Fort Irwin       CA  California       6071.0   
199         AGUANGA         Aguanga       CA  California       6065.0   

        county_name      lat       lng  population  density   source military  \
0    San Bernardino  34.4427 -116.9021    

Let's drop all columns from `iecoord_df` except `city`, `lat`, and `lng` before we proceed.

In [23]:
iecoord_df = iecoord_df.loc[:,["city","lat","lng"]]

Now lets add our IE coordinate information to `ie_df`, which will give us the final piece of information we need to add the patents to a US map. How can we achieve this? Using Pandas' simple `join` function.

If `inventor_city_name` in `ie_df` matches a `city` in `iecoord_df`, we will copy over all other column information for that entry. Because we only have `lat` and `lng` information (besides `city`) in `iecoord_df`, all will transfer over nicely into our final dataset.

In [24]:
# join respective lat,lng value for 'inventor_city_name' with 'city' value as reference
ie_df = ie_df.join(iecoord_df.set_index("city"), on="inventor_city_name")
ie_df.dropna(subset=["lat","lng"], inplace=True) # drop outlier rows if any lat,lng data is missing
print(ie_df)

      application_number inventor_name_first inventor_name_middle  \
1               04643297             WILLIAM                   R.   
4                6120673             RICHARD                   D.   
5                6139628             RAYMOND                   A.   
6                6188826             WILLIAM                   S.   
7                6200992              GORDON                   A.   
...                  ...                 ...                  ...   
47593           63053372             Douglas                  NaN   
47594           63053363             Douglas                  NaN   
47595           17375906             Douglas                  NaN   
47600           10033633              Harvey                  NaN   
47638           17480565             Montana               Marcus   

      inventor_name_last inventor_city_name inventor_region_code  \
1               MAGORIAN             RIALTO                   CA   
4                BOLDING           

We finally have finished building a clean set of data from patents in the IE! Now that the heavy lifting is done, let's export `ie_df` using Pandas' `to_csv` function!

Once this command is done, /content/export should exist! Take a peek inside and you will be able to download the dataset we have created!

In [25]:
if not os.path.exists("./export"): # make ./export if it doesn't exist
    os.mkdir("export")
ie_df.to_csv("./export/ie_patents.csv")

## **Results**

After importing into ArcGIS and mapping all rows to X,Y points, we can select "Heat Map" under symbology to view the density of patents in the surrounding areas.

![1.png](https://i.imgur.com/4QYFjHG.png)

We can setup Definition Queries to filter patents by year using the filing_date field we added

![2.png](https://i.imgur.com/igAA5Dk.png)

When selecting a city, we can see a list of patents filed that year in that city. Currently there is no support for patent title or description, but this can easily be added in the future by parsing additional datasets

![3.png](https://i.imgur.com/kdDgk2S.png)

#### Special Thanks:

- [Dylan Kai Lau](https://kailau02.github.io/Dylan-Lau-Portfolio/) for the support and advice